In [2]:
import numpy as np

# Ackley's function
def ackley(x, y):
    a = 20
    b = 0.2
    c = 2 * np.pi

    term1 = -a * np.exp(-b * np.sqrt((x**2 + y**2) / 2))
    term2 = -np.exp((np.cos(c * x) + np.cos(c * y)) / 2)
    value = term1 + term2 + a + np.exp(1)

    return value

def mutation(parent, sigma):
    # Mutate the parent by adding random noise with standard deviation sigma
    x_mutated = parent[0] + np.random.normal(0, sigma)
    y_mutated = parent[1] + np.random.normal(0, sigma)
    return (x_mutated, y_mutated)

def general_es(mu, lmbda, generations, sigma):
    # mu: Parent population size
    # lmbda: Offspring population size
    # generations: Number of generations
    # sigma: Mutation step size

    # Initialize the parent population randomly within a reasonable range
    parent_population = [np.random.uniform(low=-5, high=5, size=2) for _ in range(mu)]
    fitness_history = []  # Initialize an empty list to store fitness history

    for generation in range(generations):
        for parent in parent_population:
            # Generate lmbda offspring and evaluate their fitness
            offspring_population = [mutation(parent, sigma) for _ in range(lmbda)]
            offspring_fitness = [ackley(x, y) for x, y in offspring_population]

            # Combine parent and offspring populations
            combined_population = list(zip(parent_population, [ackley(x[0], x[1]) for x in parent_population]))
            combined_population += list(zip(offspring_population, offspring_fitness))

            # Sort the combined population based on fitness (ascending order)
            combined_population = sorted(combined_population, key=lambda x: x[1])

            # Select the best mu individuals as the new parent population
            parent_population = [x[0] for x in combined_population[:mu]]

            # Calculate the new mutation step size (adaptation)
            sigma *= 1.0 / (generation + 1)

            # Output the best individual in this generation
            best_individual = min(combined_population, key=lambda x: x[1])
            print(f"Generation {generation}: Best Individual = {best_individual[0]}, Best Fitness = {best_individual[1]}")
            
            # Append the best fitness of this generation to the fitness history
            fitness_history.append(best_individual[1])

    # Select the best individual from the final parent population
    best_individual = min(combined_population, key=lambda x: x[1])
    return best_individual[0][0], best_individual[0][1], best_individual[1], fitness_history

def one_plus_one_es(sigma_init=1.0, num_iterations=100, alpha=0.5, tol=1e-6):
    n = 2  # Number of dimensions for Ackley's function

    # Randomly initialize the parent solution
    parent = np.random.uniform(low=-5, high=5, size=n)
    sigma = sigma_init
    fitness_history = []

    for t in range(num_iterations):
        # Generate offspring by adding noise to the parent
        offspring = parent + sigma * np.random.normal(size=n)

        # Evaluate the fitness of parent and offspring
        parent_fitness = ackley(parent[0], parent[1])
        offspring_fitness = ackley(offspring[0], offspring[1])

        # Update the parent if the offspring is better
        if offspring_fitness < parent_fitness:
            parent = offspring

        # Adjust sigma (mutation strength) based on success rate
        success_rate = int(offspring_fitness < parent_fitness)
        sigma *= np.exp(alpha * (success_rate - 0.2) / 0.8)

        # Check for convergence
        if np.linalg.norm(parent) < tol:
            break
            
        # Append the best fitness of this iteration to the fitness history
        fitness_history.append(best_fitness_11)

    return parent, ackley(parent[0], parent[1]), fitness_history

def one_plus_mu_es(mu=10, sigma_init=1.0, num_iterations=100, alpha=0.5, tol=1e-6):
    n = 2  # Number of dimensions for Ackley's function

    # Randomly initialize the parent solution
    parent = np.random.uniform(low=-5, high=5, size=n)
    sigma = sigma_init
    fitness_history = []

    for t in range(num_iterations):
        # Generate µ offspring by recombination and mutation
        offspring_population = []
        for _ in range(mu):
            # Select two parents randomly from the parent and offspring populations
            parent1 = parent
            parent2 = offspring_population[np.random.randint(mu)]
            
            # Perform discrete, multi-point crossover to create offspring
            crossover_points = np.random.randint(2, size=n)
            offspring = parent1 * crossover_points + parent2 * (1 - crossover_points)
            
            # Mutate the offspring
            offspring += sigma * np.random.normal(size=n)
            
            offspring_population.append(offspring)

        # Evaluate the fitness of parent and offspring
        parent_fitness = ackley(parent[0], parent[1])
        offspring_fitnesses = [ackley(offspring[0], offspring[1]) for offspring in offspring_population]

        # Find the best µ offspring
        best_offspring_indices = np.argsort(offspring_fitnesses)[:mu]
        best_offspring_population = [offspring_population[i] for i in best_offspring_indices]
        best_offspring_fitnesses = [offspring_fitnesses[i] for i in best_offspring_indices]

        # Update the parent if the best offspring is better
        if min(best_offspring_fitnesses) < parent_fitness:
            parent = best_offspring_population[np.argmin(best_offspring_fitnesses)]

        # Adjust sigma (mutation strength) based on success rate
        success_rate = sum(f < parent_fitness for f in best_offspring_fitnesses) / mu
        sigma *= np.exp(alpha * (success_rate - 0.2) / 0.8)

        # Check for convergence
        if np.linalg.norm(parent) < tol:
            break

        # Append the best fitness of this iteration to the fitness history
        fitness_history.append(min(best_offspring_fitnesses))

    return parent, ackley(parent[0], parent[1]), fitness_history

# Run General ES
mu = 10       # Parent population size
lmbda = 50    # Offspring population size
generations = 100
sigma = 0.1   # Initial mutation step size

best_x_general, best_y_general, best_fitness_general, fitness_history_general = general_es(mu, lmbda, generations, sigma)

print("\nFinal Result (General ES):")
print(f"Best Individual: x = {best_x_general}, y = {best_y_general}")
print(f"Best Fitness: {best_fitness_general}")

# Run (1+1) ES
best_solution_11, best_fitness_11, fitness_history_11 = one_plus_one_es()
print("\nFinal Result (1+1) ES:")
print("(1+1) ES - Best solution:", best_solution_11)
print("(1+1) ES - Best fitness:", best_fitness_11)

# Run (1+µ) ES with different µ values (e.g., µ=10 and µ=50)
mu_values = [10, 50]
for mu_val in mu_values:
    best_solution_1mu, best_fitness_1mu, fitness_history_1mu = one_plus_mu_es(mu=mu_val)
    print(f"\nFinal Result (1+µ) ES (µ={mu_val}):")
    print("(1+µ) ES - Best solution:", best_solution_1mu)
    print("(1+µ) ES - Best fitness:", best_fitness_1mu)

Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = [ 0.09902723 -2.19484321], Best Fitness = 6.280665850637799
Generation 0: Best Individual = (0.0015967385525997718, -2.038994049596685), Best Fitness = 5.050482057217284
Generation 1: Best Individual = (0.030586608576347183, -1.9609

NameError: name 'best_fitness_11' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ... (Your existing code)

def plot_evolution(fitness_values, title):
    plt.figure()
    plt.plot(fitness_values, marker='o')
    #plt.scatter(range(len(fitness_values)), fitness_values, marker='o')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.title(title)
    plt.grid(True)
    plt.show()

# ... (Your existing code)

# Run General ES
mu = 10       # Parent population size
lmbda = 50    # Offspring population size
generations = 100
sigma = 0.1   # Initial mutation step size

best_x_general, best_y_general, best_fitness_general, fitness_history_general = general_es(mu, lmbda, generations, sigma)
plot_evolution(fitness_history_general, "General ES Evolution")

# Run (1+1) ES
best_solution_11, best_fitness_11, fitness_history_11 = one_plus_one_es()
plot_evolution(fitness_history_11, "(1+1) ES Evolution")

# Run (1+µ) ES with different µ values (e.g., µ=10 and µ=50)
mu_values = [10, 50]
for mu_val in mu_values:
    best_solution_1mu, best_fitness_1mu, fitness_history_1mu = one_plus_mu_es(mu=mu_val)
    plot_evolution(fitness_history_1mu, f"(1+µ) ES Evolution (µ={mu_val})")